In [8]:
import escher
import cobrakbase
import copy

In [4]:
modelseed_local = cobrakbase.modelseed.from_local('/Users/fliu/workspace/jupyter/ModelSEEDDatabase')

load: /Users/fliu/workspace/jupyter/ModelSEEDDatabase/Biochemistry/reactions.tsv
load: /Users/fliu/workspace/jupyter/ModelSEEDDatabase/Biochemistry/compounds.tsv
load: /Users/fliu/workspace/jupyter/ModelSEEDDatabase/Biochemistry/Structures/Unique_ModelSEED_Structures.txt
load: /Users/fliu/workspace/jupyter/ModelSEEDDatabase/Biochemistry/Aliases/Unique_ModelSEED_Compound_Aliases.txt
load: /Users/fliu/workspace/jupyter/ModelSEEDDatabase/Biochemistry/Aliases/Unique_ModelSEED_Reaction_Aliases.txt
load: /Users/fliu/workspace/jupyter/ModelSEEDDatabase/Biochemistry/Aliases/Unique_ModelSEED_Reaction_ECs.txt


In [5]:
cobra_metabolite = {
    'id': 'cpdXXXXX',
    'name': 'compound name',
    'compartment': 'c',
    'charge': 0,
    'formula': '',
    'annotation': {}
}
cobra_reaction = {
    'id': 'rxnXXXXX',
    'name': 'reaction name',
    'metabolites': {'cpdXXXX1': -1.0,
                    'cpdXXXX2': 1.0,
                    'cpdXXXX3': 1.0
                   },
    'lower_bound': -1000.0,
    'upper_bound': 1000.0,
    'gene_reaction_rule': '',
    'annotation': {}
}

In [ ]:
rxn_build_template = {}
for rxn_id in modelseed_local.reactions:
    rxn = modelseed_local.get_seed_reaction(rxn_id)
    cmps = set(map(lambda x : x[1], rxn.cstoichiometry))
    if not rxn.is_obsolete:
        if len(cmps) == 1:
            rxn_build_template[rxn.id] = [{'0' : 'c', '1' : 'c'}]
        elif len(cmps) == 2:
            rxn_build_template[rxn.id] = [{'0' : 'e', '1' : 'c'}, {'0' : 'c', '1' : 'e'}]
        else:
            #print(rxn.id)
            pass

In [32]:
def build_model_from_rxn_ids(ms, rxn_build_template):
    all_compounds = []
    all_reactions = []
    all_reaction_compounds = {}
    for rxn_id in rxn_build_template:
        rxn = ms.get_seed_reaction(rxn_id)
        balanced = rxn.data['status']
        if balanced == 'OK':
            cmps = set(map(lambda x : x[1], rxn.cstoichiometry))
            for cmp_template in rxn_build_template[rxn_id]:
                rxn_cmp = cmp_template['0']
                cobra_reaction, cpd_cmp = modelseed_to_cobra_reaction(rxn, cmp_template, rxn_cmp)
                for cpd_id in cpd_cmp:
                    if not cpd_id in all_reaction_compounds:
                        all_reaction_compounds[cpd_id] = set()
                    for cmp_id in cpd_cmp[cpd_id]:
                        all_reaction_compounds[cpd_id].add(cmp_id)

                all_reactions.append(cobra_reaction)
            
    for cpd_id in all_reaction_compounds:
        cpd = ms.get_seed_compound(cpd_id)
        for cmp_id in all_reaction_compounds[cpd_id]:
            all_compounds.append(modelseed_to_cobra_compound(cpd, cmp_id))
            
    logger.warning('metabolites (unique): %d, reactions: %d', len(all_reaction_compounds), len(all_reactions))
            
    base_model = {
        'id' : 'ModelSEED Generic',
        'version' : '1',
        'compartments' : {'z' : 'any'},
        'metabolites' : all_compounds,
        'reactions' : all_reactions,
        'genes' : [],
        'annotation' : {}
    }
            
    return base_model

base_model = build_model_from_rxn_ids(modelseed_local, rxn_build_template)

NameError: name 'rxn_build_template' is not defined

In [31]:
cmp_id

NameError: name 'cmp_id' is not defined

In [77]:
all_compounds = []
all_reactions = []
all_reaction_compounds = {}
for rxn_id in modelseed_local.reactions:
    rxn = modelseed_local.get_seed_reaction(rxn_id)
    if not rxn.is_obsolete and not rxn.is_transport:
        cobra_reaction, cpd_cmp = modelseed_to_cobra_reaction(rxn, {'0' : None, '1' : None}, None)
        all_reaction_compounds.update(cpd_cmp)
        all_reactions.append(cobra_reaction)
        
        
for cpd_id in all_reaction_compounds:
    cpd = modelseed_local.get_seed_compound(cpd_id)
    for cmp_id in all_reaction_compounds[cpd_id]:
        all_compounds.append(modelseed_to_cobra_compound(cpd, cmp_id))

base_model = {
    'id' : 'ModelSEED Generic',
    'version' : '1',
    'compartments' : {'z' : 'any'},
    'metabolites' : all_compounds,
    'reactions' : all_reactions,
    'genes' : [],
    'annotation' : {}
}

In [53]:
for m in base_model['metabolites']:
    if not type(m['name']) == str:
        print(type(m['name']))
for m in base_model['reactions']:
    if not type(m['name']) == str:
        print(type(m['name']))

In [78]:
escher_builder = escher.Builder(model_json=json.dumps(base_model))

In [80]:
#escher_builder.display_in_notebook(enable_editing=True)

In [82]:
escher_builder.save_html('../index.html', overwrite=True)

'../index.html'

In [76]:
def modelseed_to_cobra_compound(cpd, cmp = 'z'):
    ccpd = copy.deepcopy(cobra_metabolite)
    ccpd['id'] = cpd.id
    ccpd['compartment'] = 'z'
    if not cmp == None:
        ccpd['id'] = cpd.id + '_' + cmp
        ccpd['compartment'] = cmp
    ccpd['name'] = cpd.data['name']
    ccpd['formula'] = ''# if cpd.data['formula'] cpd.data['formula']
    ccpd['charge'] = cpd.data['charge']
    
    
    
    return ccpd


def modelseed_to_cobra_reaction(rxn, compartment = {'0' : None}, rxn_cmp_suffix = 'c', lb = None, ub = None):
    crxn = copy.deepcopy(cobra_reaction)
    crxn['id'] = rxn.id
    if not rxn_cmp_suffix == None:
        crxn['id'] = rxn.id + '_' + rxn_cmp_suffix
    crxn['name'] = rxn.id
    
    metabolites = {}
    cpd_cmp = {}
    cs = rxn.cstoichiometry
    #print(cs)
    for p in cs:
        value = cs[p]
        met_id = p[0]
        if not p[1] in compartment:
            print(rxn.data)
        cmp = compartment[p[1]]
        if not cmp == None and not len(cmp.strip()) == 0:
            met_id += '_' + cmp.strip()
        elif cmp == None or len(cmp.strip()) == 0:
            cmp = None
        else:
            cmp = 'z'
        if not met_id in metabolites:
            metabolites[met_id] = value
            if not p[0] in cpd_cmp:
                cpd_cmp[p[0]] = set()
            cpd_cmp[p[0]].add(cmp)
        else:
            print('!', rxn.id)
    
    rev = rxn.data['direction'] #direction
    if lb == None and ub == None:
        
        lb = -1000
        ub =  1000
        #print('reversibility', rxn.data['reversibility'], 'direction', rxn.data['direction'])
        if rev == '>':
            lb = 0
        elif rev == '<':
            ub = 0
    crxn['lower_bound'] = lb
    crxn['upper_bound'] = ub
    crxn['metabolites'] = metabolites
    return crxn, cpd_cmp
#rxn = modelseed_local.get_seed_reaction('rxn08071')
#print(rxn.cstoichiometry)
#modelseed_to_cobra_reaction(rxn, {'0' : ''})
cobra_reaction, cpd_cmp = modelseed_to_cobra_reaction(rxn, {'0' : None, '1' : None}, None)
cobra_reaction

{'id': 'rxn48575',
 'name': 'rxn48575',
 'metabolites': {'cpd37299': -2.0, 'cpd00794': 1.0, 'cpd02725': 1.0},
 'lower_bound': -1000,
 'upper_bound': 1000,
 'gene_reaction_rule': '',
 'annotation': {}}

In [20]:
for rxn_id in modelseed_local.reactions:
    rxn = modelseed_local.get_seed_reaction(rxn_id)
    if not rxn.is_obsolete and not rxn.is_transport:
        modelseed_to_cobra_reaction(rxn, {'0' : '', '1' : ''})